# Task 1

# Web scraping

In [16]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import requests

header = {'User-Agent': 'NodePing'}
url='https://www.zomato.com/bangalore/' #link of we site 
response= requests.get(url,headers=header)
soup=BeautifulSoup(response.content,'lxml')
Zomato_area_link=[]
def resutrent_area(soup):
    links=[]
    for link in soup.find_all('a'):
        links.append(link.get('href')) #extrating href tags from soup
    
    links = list(filter(None, links)) 

    for i in range(len(links)): # finding all resturent link with help of regex
        if re.findall(r"^https:.*-restaurants$",links[i]):
            Zomato_area_link.append(links[i]) # final list 
            
resutrent_area(soup)

In [17]:
Zomato_rest_link=[]
def resutrent_name(i,header):
    links=[]
    rest_link=[]
    page = requests.get(i, headers=header)
    soup = BeautifulSoup(page.text,'lxml')
    for link in soup.find_all('a'):
            links.append(link.get('href'))
            
    links = list(filter(None, links)) 

    for i in range(len(links)):
        if re.findall(r"order$",links[i]):
            rest_link.append(links[i])
            
    Zomato_rest_link.extend(rest_link)
            
for i in Zomato_area_link:
    header = {'User-Agent': 'NodePing'}
    resutrent_name(i,header)

Zomato_rest_link=(list(set(Zomato_rest_link)))
Zomato_rest_link = list(map(lambda x:"https://www.zomato.com"+x,Zomato_rest_link))

In [28]:
data=pd.DataFrame()
c=0
def list_all(links):
    cuisine_name=[]
    price=[]
    description=[]
    rest_name=[]

    global data
    
    page = requests.get(links, headers=header)
    soup = BeautifulSoup(page.text, 'lxml')
    
    rest_name.append(soup.select('h1.sc-7kepeu-0')[-1].text)

    for h in soup.find_all('h4',class_="sc-1s0saks-15 iSmBPS"):
        cuisine_name.append(h.text)
        
    rest_name=rest_name*len(cuisine_name)
    
    for h1 in soup.find_all('span',class_="sc-17hyc2s-1 cCiQWA"):
        price.append(h1.text)

    for h2 in soup.find_all('p',class_="sc-1s0saks-12 hcROsL"):
        description.append(h2.text)
    
    
    if data.empty:
        data=pd.DataFrame(zip(rest_name,cuisine_name,price,description))
    else:
        data1=pd.DataFrame(zip(rest_name,cuisine_name,price,description))
        data=data.append(data1,ignore_index = True)

for i in Zomato_rest_link:
    list_all(i)
    
data.to_csv('Zomata_menu1.csv',index=False) 

In [2]:
import pandas as pd
a=pd.read_csv('Zomata_menu.csv')
a.rename(columns={'0':'rest_name','1':'cuisine_name','2':'price','3':'description'},inplace=True)
a.to_csv("zomato_m.csv")

In [8]:
a.shape

(21545, 4)

# Task 2

# Come up with the frequency of each word used across all item descriptions

In [4]:
a['description']

0        Empire classic batter fried Kebab, Signature d...
1        Clay pot oven grilled chicken with a smoky fla...
2          Succulent Chiken Meat in aromatic Biriyani Rice
3                       Buttery tomato gravy with chicken.
4          Choice of any non-veg Starter + Chicken Biryani
                               ...                        
21540                           Mango juice, coconut, mint
21541                                                  NaN
21542                                                  NaN
21543                                                  NaN
21544                                                  NaN
Name: description, Length: 21545, dtype: object